In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import multiprocessing
from multiprocessing.pool import ThreadPool

import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import tqdm
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
from data_io import load_data_by_ticker, get_call_data_for_given_ticker

In [4]:
calls = pd.read_pickle('data/CALLS_TRANSFORMED.pkl')
kospi_downloaded = pd.read_pickle('history/kospi_download_result.pkl')
kospi = pd.read_csv('metadata/kospi.csv', error_bad_lines=False)

b'Skipping line 437: expected 12 fields, saw 13\n'


In [5]:
tickers = kospi_downloaded['종목코드']

In [6]:
_, columns_for_df = get_call_data_for_given_ticker(calls, kospi, tickers[0])
columns_for_df = columns_for_df.columns.insert(0, 'Time lag').insert(0, 'Correlation with').insert(0, 'ticker').insert(0, '기업명')
df_corr = pd.DataFrame(columns=columns_for_df)
df_corr

,기업명,ticker,Correlation with,Time lag,High,Low,Open,Close,Volume,Adj Close,...,"중랑구, 20대, 남","중랑구, 20대, 여","중랑구, 30대, 남","중랑구, 30대, 여","중랑구, 40대, 남","중랑구, 40대, 여","중랑구, 50대, 남","중랑구, 50대, 여","중랑구, 60대, 남","중랑구, 60대, 여"


In [7]:
def get_correlation_by_ticker(calls, kospi, ticker, idx) : 

    company_name, df_totest = get_call_data_for_given_ticker(calls, kospi, ticker)
    
    
    time_lags = [0,1,2,5,10,25,50]
    
    # for time lag
    for idx_time_lag, time_lag in enumerate(time_lags) : 
        df = df_totest.copy()
        if time_lag != 0 :
            df[['High', 'Low', 'Open', 'Close', 'Volume']] = df[['High', 'Low', 'Open', 'Close', 'Volume']].shift(periods=time_lag*(-1))
        
        # for price and volume
        candidates = ['High', 'Low', 'Open', 'Close', 'Volume']
        for idx_candidate, candidate in enumerate(candidates) : 
            temp = df.corr()[candidate]
            temp = pd.concat([pd.Series([company_name, ticker, candidate, time_lag], index=['기업명', 'ticker', 'Correlation with', 'Time lag']), temp])
            df_corr.loc[idx*(len(time_lags)*len(candidates))+idx_time_lag*len(candidates)+idx_candidate] = temp

    pbar.update(1)
    return

#idx = 0
#get_correlation_by_ticker(calls, kospi, tickers[idx], idx)
#df_corr.head()

In [ ]:
pool = ThreadPool(24)
with tqdm.tqdm(total=tickers.shape[0]) as pbar :
    for idx in range(tickers.shape[0]) : 
        pool.apply_async(get_correlation_by_ticker, [calls, kospi, tickers[idx], idx])
        
    pool.close()
    pool.join()
    
df_corr.to_pickle('metadata/df_corr.pkl')

 37%|███▋      | 290/788 [26:01<58:39,  7.07s/it]  